<h1 style="text-align:center">Travail pratique numérique en thermodynamique statistique</h1>
<h2 style="text-align:center">PARTIE 1 : Cinétique des gaz parfaits</h2>

Veuillez indiquer le nom des membres de votre équipe dans la cellule suivante.

- Michael Hudon
- Émile Poirier
- Justin Binette
- Sébastien Rouleau

# Introduction #
Ce travail révise d'abord quelques bases générales de mécanique statistique classique avec le script `tds2Danimation_hXX.py` qui simule la théorie cinétique des gaz parfaits en 2D. Cette simulation utilise le concept de sphères dures, mais ici pour le cas des particules d'un gaz afin d'introduire des collisions élastiques entre elles sur leur trajectoire ballistique. Notez qu'une sphère est colorée et grossie seulement pour l’effet visuel dans l'animation, la physique de l’algorithme codé considère bien des particules totalement identiques. Les questions sur cette simulation, à répondre directement dans les cellules du carnet _(Notebook)_ ici-même, explorent quelques paramètres de la thermodynamique statistique et introduisent de nouveaux termes utiles à l'étude du mouvement des électrons dans la matière.

_N.B._ 
- _Pour montrer les animations à l'écran, le script `tds2Danimation_hXX.py` importe la librairie `VPython` qu'il faut donc installer. Des liens vers sa documentation et de l'information complémentaire sont donnés dans la médiagraphie à la fin._
- _Le code dans ce script est abusivement commenté dans notre contexte pédagogique, mais il serait bien sûr préférable de s’en tenir aux recommandations du <a href="https://www.python.org/dev/peps/pep-0008"> PEP 8 — Style Guide for Python Code</a>._
- _Notez finalement que la boucle principale à la fin du script laisse l'utilisateur voir l'animation aussi longtemps que souhaité, assurez-vous donc de savoir comment l'interrompre correctement avant de lancer la simulation ou de la remplacer par une boucle `for`._

# 1<sup>re</sup> partie - Cinétique CLASSIQUE des gaz parfaits #

### Simulation 2D ###

In [2]:
import os
import sys


os.chdir('C:/Users/EmilePc/Documents/GitHub/TP_Num_Thermo_Stat')    #   Mettre le path de ou ce trouve les fichiers Jupyter.

In [26]:
!python tds2Danimation_h25.py
# Remplacez "XX" par les deux derniers chiffres de l'année de votre session.
# N'hésitez pas à exécuter l'animation 2D ou la simulation 3D à l'extérieur du _Notebook_. Cette cellule vise à préciser que les questions qui suivent se basent sur ces scripts et doivent mener aux mêmes réponses autant en 2D qu'en 3D.

Données sauvegardées dans le fichier C:\Users\EmilePc\Documents\GitHub\TP_Num_Thermo_Stat\p_data.json


### Questions statistiques ###

**I.** _(3 points)_  &mdash; Utilisez la liste finale des vecteurs de quantité de mouvement $\vec{p}$ de toutes les sphères pour calculer la moyenne de son carré $\langle p^2\rangle=\langle\vec{p}\cdot\vec{p}\rangle$ en écrivant le code nécessaire dans la cellule qui suit. 

In [3]:
import json
import numpy as np

# Charger les données du fichier JSON (généralisation pour 2D)
def get_quantite_mouvement(filename="p_data.json"):
    with open(filename, 'r') as f:
        data = json.load(f)
    # Retourne toutes les dimensions disponibles (x, y)
    return [[item.get('x', 0), item.get('y', 0)] for item in data]

p = get_quantite_mouvement()

p_x = [item[0] for item in p]  # Liste des composantes x
p_y = [item[1] for item in p]  # Liste des composantes y

p_carre = [x ** 2 + y ** 2 for x, y in zip(p_x, p_y)]

# Calculer la moyenne de p^2
p2_moyenne = np.mean(p_carre)

# Afficher le résultat
print(f"La moyenne de p^2 est : {p2_moyenne}")

La moyenne de p^2 est : 5.6e-47



**II.** _(2 points)_  &mdash; La température $T$ (macroscopique) est proportionnelle à l'énergie cinétique moyenne $E_{cin}$ de l'ensemble des particules lorsque ce système est rendu à l'équilibre. Celle-ci peut se calculer classiquement selon son <a href="https://fr.wikipedia.org/wiki/%C3%89quipartition_de_l%27%C3%A9nergie">principe d'équipartition</a>, _i.e._ en répartissant l'énergie également sur chaque degré de liberté ici en translation seulement d'où, au total pour $\text{DIM}=1,2\text{ ou } 3$ dimensions d'espace réel,
\begin{equation}
E_{cin}=\frac{\langle p^2 \rangle}{2m}=\text{DIM}\times\frac{1}{2}k_BT
\end{equation}
avec $k_B$, la constante de Boltzmann et $m$, la masse de chaque particule. Quelle est la température du gaz de sphères dures à la fin de la simulation? Est-ce qu'elle a changé significativement par rapport à sa valeur initiale?

In [4]:
DIM = 2 #Nombre de degrés de liberté de la simulation
mass = 4E-3/6E23 # helium mass
k = 1.4E-23 # Boltzmann constant
T = 300 #Température début

def Ecin(p2, m):
    return p2_moyenne / (2 * m)

def Temp(Ecin_moyenne):
    return (2*Ecin_moyenne)/(DIM*k)

T_final = Temp(Ecin(p2_moyenne, mass))

print(f"La température du gaz de sphères dures au début et à la fin de la simulation sont: {T}K et {T_final}K")

La température du gaz de sphères dures au début et à la fin de la simulation sont: 300K et 300.00000000000006K


**III.** _(10 points)_ &mdash; Modifiez le code de la simulation pour ajouter une fonction qui suit la trajectoire d'UNE SEULE particule, c'est-à-dire qu'elle doit enregistrer, dans une liste, des valeurs de variables pour cette particule et ce, à chacune de ses collisions avec une autre particule (_i.e._ excluez les collisions avec les parois de la boîte). Les deux variables scalaires à lister sont:
- la distance que la particule a parcouru entre chaque collision,
- le temps écoulé entre ces collisions.

Copiez le code de votre fonction dans la cellule qui suit en y commentant clairement les variables pour ces listes qui devront persister après avoir interrompu l'exécution de la simulation. N'oubliez pas d'inclure votre fichier Python (`.py`) modifié avec la simulation complète lors de la remise.

In [4]:
# Choix de la particule suivit entre chaque collision.
# Ligne du code =

id_particule = 1

#-------------------------------------------------------------------------------------------------------------------------------------------------

# Met en rouge la particule suivit.
# Ligne du code =

if i == id_particule:
    Atoms.append(simple_sphere(pos=vector(x,y,z), radius=Ratom, color=color.red))

#-------------------------------------------------------------------------------------------------------------------------------------------------

# Initialisation des listes finale de distance et de temps.
# Ligne du code =

distance_collision = []
temps_collision = []

#-------------------------------------------------------------------------------------------------------------------------------------------------

# Définition de la fonction permettant de détecter la distance parcourue et le temps entre chaque collision.
# Ligne du code =

def suivre_particule(id, hitlist, vitesse, step, dt):

    # Initialisation
    position_particule = []
    distance_collision_particule = []
    temps_collision_particule = []
    current_time = step * dt  # Calcule le temps accumulé jusqu'à l'étape

    # Ajoute la position actuelle de la particule suivie
    position_particule.append(apos[id])  # Enregistre la position actuelle de la particule suivie

    # Vérifie si la particule suivie est impliquée dans une collision
    if hitlist:  # S'il y a des collisions détectées
        for paire in hitlist:  # Parcourt les collisions
            i, j = paire  # Indices des particules en collision

            if i == id or j == id:  # Si la particule suivie est impliquée dans une collision
                distance_parcourue = vitesse[id].mag * current_time  # Calcule la distance parcourue
                distance_collision_particule.append(distance_parcourue)  # Stocke la distance parcourue
                temps_collision_particule.append(current_time)  # Stocke le temps écoulé

                current_time = 0  # Réinitialise le temps pour cet intervalle

    return distance_collision_particule, temps_collision_particule

#-------------------------------------------------------------------------------------------------------------------------------------------------

# Appel de la fonction dans la boucle For de la simulation.
# Ligne du Code =

distance_collision_particule, temps_collision_particule = suivre_particule(id_particule, hitlist, vitesse, step, dt)
distance_collision.extend(distance_collision_particule)     # VARIABLE QUI RESTE CONTENANT LA LISTE DES DISTANCES ENTRE LES COLLISIONS.
temps_collision.extend(temps_collision_particule)       # VARIABLE QUI RESTE CONTENANT LA LISTE DES TEMPS ENTRE LES COLLISIONS.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

**IV.** _(2 points)_ &mdash; Calculez le **libre parcours moyen** $l_{moy}$ et le **temps de collision** $\tau$ qui sont les valeurs moyennes des deux listes compilées au numéro précédent.

_(Pour votre information, le libre parcours moyen est de l’ordre de 100 nm dans l'air à température et pression ambiantes, mais_ $l_{moy}$ _peut dépasser 100 000 km dans une enceinte sous vide avec les technologies de pompes modernes!)_



In [7]:
def get_distance_temps(filename="distance_temps_collision_data.json"):
    with open(filename, 'r') as f:
        data = json.load(f)
    # Retourne toutes les dimensions disponibles (x, y)
    return [[item.get('distance', 0), item.get('temps', 0)] for item in data]

distance_temps = get_distance_temps()

distance = [item[0] for item in distance_temps]  # Liste des composantes x
temps = [item[1] for item in distance_temps]  # Liste des composantes y

l_moy = np.mean(distance)
tau = np.mean(temps)

print(f"Le libre parcours moyen est :{l_moy} mètre et le temps de collision est : {tau} seconde.")

Le libre parcours moyen est :4.17201702860735 mètre et le temps de collision est : 0.004452838427947598 seconde.


**V.** _(2 points)_ Calculez la vitesse $\vec{v}$ de la particule entre chaque paire de collisions. Quelle est la vitesse moyenne $\langle\vec{v}\rangle$ de la particule?

In [21]:
distance_1 = []
distance_2 = []

for i, val in enumerate(distance):
    if i % 2 == 0:
        distance_1.append(val)
    else:
        distance_2.append(val)

temps_1 = []
temps_2 = []

for i, val in enumerate(temps):
    if i % 2 == 0:
        temps_1.append(val)
    else:
        temps_2.append(val)

vitesses = []

for d1, d2, t1, t2 in zip(distance_1, distance_2, temps_1, temps_2):
    if t1 != t2:  # Avoid division by zero
        vitesses.append((d1 - d2) / (t1 - t2))

print(f"La vitesse moyenne de la particule en valeur absolu est : {abs(np.mean(vitesses))} m/s")


La vitesse moyenne de la particule en valeur absolu est : 3835.1746073850254 m/s


**VI.** _(5 points)_ &mdash; Pour cette même liste de vitesses, comparez les distributions de la norme $||\vec{v}||$, du carré $v^2$ et d’une de ses composantes $v_x^2$ en étalonnant l’abscisse pour contraster les histogrammes avec une échelle appropriée. Indiquez sur ce graphique la moyenne, le mode, la médiane et la moyenne quadratique des distributions.

In [ ]:

#

**Bonus.** _(4 points)_ &mdash; Montrez que 
- (a) le théorème central limite est satisfait par une des distributions de vitesse du numéro précédent,
- (b) le système simulé est ergodique.

In [ ]:

#

# Médiagraphie #
 - La simulation utilise la librairie <a href="https://vpython.org">VPython</a> conçue pour la visualisation de physique en 3D, avec les instructions d’installation <a href="https://vpython.org/presentation2018/install.html">ici</a> et la documentation <a href="https://www.glowscript.org/docs/VPythonDocs/index.html">ici</a>. Le script fourni qui est exécuté dans ce _Notebook_ adapte en 2D et commente en détail l’exemple <a href="https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/HardSphereGas-VPython">HardSphereGas-VPython</a> du site interactif <a href="https://www.glowscript.org">GlowScript</a> pour programmer des animations avec VPython directement en ligne.
 
_**N.B. de débogage VPython:** Ayant plus codé en MATLAB qu'en Python jusqu'à maintenant, j'utilise Spyder qui offre une interface similaire, mais j'ai trouvé ce vidéo qui parle d'une <a href="https://www.youtube.com/watch?v=MJiVtz4Uj7M">installation VS code</a> qui peut-être aider? N'hésitez pas à partager de meilleures alternatives que j'ajouterai ici. Vous pouvez aussi tenter l'installation <a href="https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/">directement dans le Notebook</a>, mais au final chaque installation de distribution Python a son lot de <a href="https://xkcd.com/2347/">défis avec les dépendances</a>. Si rien ne fonctionne, n'hésitez pas à contacter la sympathique communauté qui développe et utilise cette librairie VPython d'abord via leur <a href="https://groups.google.com/g/vpython-users">groupe-forum</a>, puis leur <a href="https://github.com/vpython/vpython-jupyter">site GitHub</a> de dépôt de développement. J'ai toujours eu de bonnes expériences d'interaction avec eux!_ 
 